Задание: Бейзлайн модель, которая работает на отложенной выборке (train_test_split) и validation.csv (различие в точности может быть большим, но главное попробовать любыми способами его уменьшить и зафиксировать эксперименты) - 3 балла

Воспользуюсь переводом валидации на английский.
Буду перебирать признаки  и печатать результаты тестирования на отложенной выборке, результате перевода валидации, полученном кодом:

from googletrans import Translator
translator = Translator()
validation['translation'] = validation['comment_text'].apply(lambda x: translator.translate(x).text)
validation.to_csv('validation_with_translation.csv')

Часть текстов не перевелась, поэтому отдельно буду считать на абсолютно всех текстах и только на переведенных

In [57]:
import pandas as pd
import sklearn
train1 = pd.read_csv('toxic-comment-train.csv')
validation = pd.read_csv('validation_with_translation.csv')

In [10]:
train1

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [58]:
validation_really_translated = validation.query('comment_text != translation')

In [59]:
from sklearn.linear_model import LogisticRegression

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
tf = TfidfVectorizer()

In [62]:
model = LogisticRegression()

In [63]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
tf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [65]:
texts = train1.comment_text
y = train1.toxic
texts_train, texts_test, y_train, y_test = sklearn.model_selection.train_test_split(texts, y)

texts_valid_all = validation.comment_text
y_valid_all = validation.toxic

texts_valid_translated = validation_really_translated.comment_text
y_valid_translated = validation_really_translated.toxic

In [28]:
len(texts_valid_translated)

5226

Так как я хочу не просто найти лучшие метрики, а посмотреть, как соотносятся результаты на трёх разных тестах, перебор буду писать вручную.

In [77]:
for ngram_range in [(1, 1), (1,3)]:
    for max_features in [750, 1500]:
        tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
        print(tfidf)
        X_train = tfidf.fit_transform(texts_train)
        X_test = tfidf.transform(texts_test)
        X_valid_all= tfidf.transform(texts_valid_all)
        X_valid_translated = tfidf.transform(texts_valid_translated)
        for C in [0.001, 0.5, 1, 5, 10, 100, 500, 1000]:
            print('C='+str(C))
            model = LogisticRegression(C=C)
            model.fit(X_train, y_train)
            english_prediction = model.predict_proba(X_test)[:,1]
            all_valid_prediction = model.predict_proba(X_valid_all)[:,1]
            translated_valid_prediction = model.predict_proba(X_valid_translated)[:,1]
            english_roc = sklearn.metrics.roc_auc_score(y_test, english_prediction)
            all_valid_roc = sklearn.metrics.roc_auc_score(y_valid_all, all_valid_prediction)
            translated_valid_roc = sklearn.metrics.roc_auc_score(y_valid_translated, translated_valid_prediction)
            print('english: ', english_roc, 'all_v: ', all_valid_roc, 'translated_v: ', translated_valid_roc)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=750,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
C=0.001


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.8531801962710996 all_v:  0.5724119441342124 translated_v:  0.5634333039841515
C=0.5


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.9291756179399506 all_v:  0.6051519136313963 translated_v:  0.5966380970724191
C=1


...

english:  0.930119397057605 all_v:  0.5901510730026059 translated_v:  0.5810495542592999
C=500


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.9299021119615268 all_v:  0.590039929867541 translated_v:  0.5808790997138455
C=1000


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.9298452097945114 all_v:  0.5899560471232481 translated_v:  0.5807756438476777


При С близком к нулю результаты  ужасные, при чуть-чуть больших значениях - возрастают и дальше не сильно меняются при тех же настройках векторайзера. На всей валидации результат чуть-чуть лучше, чем на выбранной только переведенной.
На переводе работает сильно хуже, чем на отложенной выборке. Попробую прогнать трейн через переводчик туда-обратно в надежде унифицировать используемую лексику

In [41]:
import googletrans
import random

In [48]:
from googletrans import Translator
translator = Translator()

In [49]:
def double_translation(english_text):
    if random.random() > 0.5:
        dest = 'de'
    else:
        dest = 'fr'
    translated = translator.translate(text=english_text, dest=dest).text
    new_english_text = translator.translate(translated).text
    return new_english_text

К сожалению, это не получилось запустить на всех данных, возьму сампл

In [47]:
sample = train1[:5000]

In [50]:
sample['bad_english'] = sample['comment_text'].apply(double_translation)

C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
sample.to_csv('train_with_double_translation.csv')

In [79]:
sample = pd.read_csv('train_with_double_translation.csv')

In [80]:
sample

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,bad_english
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation\nWhy were the changes made under m...
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"D'aww! It matches this background color, which..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"Hey man, I'm not really trying to change the w..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"""\nMore\nI can't make any real suggestions for..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"You, sir, are my hero. Do you remember the pag..."
...,...,...,...,...,...,...,...,...,...,...
4995,4995,0d39d2eaa40a6241,"""\n\nHello Marcruhwedell, and Welcome to Wikip...",0,0,0,0,0,0,"""\n\nHello Marcruhwedell and welcome to Wikipe..."
4996,4996,0d3b57f3b2db3f03,"...that's why I did ....cheers, (talk · cont...",0,0,0,0,0,0,"... that's why I have ... cheers, (discussion ..."
4997,4997,0d3bb1f12d90a6a2,"No, it's not a delayed reaction\n\nI just happ...",1,0,0,0,1,0,"No, this is not a delayed reaction\n\nI just r..."
4998,4998,0d3be13abf1bec52,"""\n\nA slight difference with you\nI have to d...",0,0,0,0,0,0,"""\n\nA slight difference with you\nI have to d..."


Пересчитаю на оригинале на сампле, чтобы корректно сравнивать

In [81]:
train, test = sklearn.model_selection.train_test_split(sample)

original_train = train.comment_text
original_test = test.comment_text

bad_english_train = train.bad_english
bad_english_test = test.bad_english

y_train = train.toxic
y_test = test.toxic

texts_valid_all = validation.comment_text
y_valid_all = validation.toxic

texts_valid_translated = validation_really_translated.comment_text
y_valid_translated = validation_really_translated.toxic

In [82]:
for ngram_range in [(1, 1), (1,3)]:
    for max_features in [750, 1500]:
        tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
        print(tfidf)
        X_train = tfidf.fit_transform(original_train)
        X_test = tfidf.transform(original_test)
        X_valid_all= tfidf.transform(texts_valid_all)
        X_valid_translated = tfidf.transform(texts_valid_translated)
        for C in [0.001, 0.5, 1, 5, 10, 100, 500, 1000]:
            print('C='+str(C))
            model = LogisticRegression(C=C)
            model.fit(X_train, y_train)
            english_prediction = model.predict_proba(X_test)[:,1]
            all_valid_prediction = model.predict_proba(X_valid_all)[:,1]
            translated_valid_prediction = model.predict_proba(X_valid_translated)[:,1]
            english_roc = sklearn.metrics.roc_auc_score(y_test, english_prediction)
            all_valid_roc = sklearn.metrics.roc_auc_score(y_valid_all, all_valid_prediction)
            translated_valid_roc = sklearn.metrics.roc_auc_score(y_valid_translated, translated_valid_prediction)
            print('english: ', english_roc, 'all_v: ', all_valid_roc, 'translated_v: ', translated_valid_roc)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=750,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
C=0.001
english:  0.7112995766488414 all_v:  0.5049893720502936 translated_v:  0.509232335461149
C=0.5
english: 

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


 0.8813154523172905 all_v:  0.5733143591406371 translated_v:  0.5640035494166851
C=1
english:  0.8848874777183601 all_v:  0.5709492500390292 translated_v:  0.5626348228043143
C=5
english:  0.8823042000891266 all_v:  0.5663880582675842 translated_v:  0.5583254457407
C=10
english:  0.877026236631016 all_v:  0.5619554226561467 translated_v:  0.5535547820823244
C=100


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.8512979055258467 all_v:  0.5564295493028786 translated_v:  0.5437826051067576
C=500
english:  0.8313976158645278 all_v:  0.5529231064836498 translated_v:  0.5398865012106537
C=1000


...

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

english:  0.8733567290552586 all_v:  0.5809824548762474 translated_v:  0.5735701904028175
C=100
english:  0.8501838235294118 all_v:  0.5877108477140902 translated_v:  0.5800876348228043
C=500


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.8410204991087346 all_v:  0.5925957416147278 translated_v:  0.5856494882236407
C=1000
english:  0.8375111408199643 all_v:  0.594409698454444 translated_v:  0.5871135538190624


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Результаты того же порядка, соотношение такое же: сильно лучше на оригинале, на всей валидации немного лучше, чем только на переведенной

In [83]:
for ngram_range in [(1, 1), (1,3)]:
    for max_features in [750, 1500]:
        tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
        print(tfidf)
        X_train = tfidf.fit_transform(bad_english_train)
        X_test = tfidf.transform(bad_english_test)
        X_original = tfidf.transform(original_test)
        X_valid_all= tfidf.transform(texts_valid_all)
        X_valid_translated = tfidf.transform(texts_valid_translated)
        for C in [0.001, 0.5, 1, 5, 10, 100, 500, 1000]:
            print('C='+str(C))
            model = LogisticRegression(C=C)
            model.fit(X_train, y_train)
            bad_english_prediction = model.predict_proba(X_test)[:,1]
            original_english_prediction = model.predict_proba(X_original)[:,1]
            all_valid_prediction = model.predict_proba(X_valid_all)[:,1]
            translated_valid_prediction = model.predict_proba(X_valid_translated)[:,1]
            bad_english_roc = sklearn.metrics.roc_auc_score(y_test, bad_english_prediction)
            original_english_roc = sklearn.metrics.roc_auc_score(y_test, original_english_prediction)
            all_valid_roc = sklearn.metrics.roc_auc_score(y_valid_all, all_valid_prediction)
            translated_valid_roc = sklearn.metrics.roc_auc_score(y_valid_translated, translated_valid_prediction)
            print('original_english:', original_english_roc,'bad_english: ', bad_english_roc, 'all_v: ', all_valid_roc, 'translated_v: ', translated_valid_roc)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=750,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
C=0.001
original_english: 0.6936622660427807 bad_english:  0.6683412711675579 all_v:  0.4900229371569934 translated_v:  0.4970945685670262
C=0.5
original_english: 0.8557263814616757 bad_english:  0.8368879512032087 all_v:  0.5602954209748893 translated_v:  0.5525293033237948
C=1
original_english: 0.8598624108734403 bad_english:  0.8412607230392158 all_v:  0.5577977927489762 translated_v:  0.5501107473035439
C=5
original_english: 0.8506573083778965 bad_english:  0.8335944463012479 all_v:  0.55673391696

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

 0.8407559046345812 bad_english:  0.8229967413101604 all_v:  0.5568070516746526 translated_v:  0.5495073464670923
C=100
original_english: 0.8097078319964349 bad_english:  0.7926797849821746 all_v:  0.5535803581078647 translated_v:  0.5428374697336562
C=500
original_english:

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


 0.8025916332442068 bad_english:  0.7823814895276292 all_v:  0.5484118120354025 translated_v:  0.5367195410521682
C=1000
original_english: 0.8010388814616756 bad_english:  0.7802229556595366 all_v:  0.5462712709106412 translated_v:  0.5345010180497469
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1500,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


...

original_english: 0.771338151737968 bad_english:  0.7602599988859179 all_v:  0.5469763783309916 translated_v:  0.5347878604446401
C=1000
original_english: 0.7645492145721925 bad_english:  0.7534432096702318 all_v:  0.5479189633846117 translated_v:  0.5369530046224962
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1500,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
C=0.001
original_english: 0.7158882018716577 bad_english:  0.6882937834224598 all_v:  0.4978592787404979 translated_v:  0.5039756218357914
C=0.5
original_english: 0.8763995655080213 bad_english:  0.8501420454545455 all_v:  0.583453543250

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


 0.8751949643493762 bad_english:  0.8519733177361853 all_v:  0.5763039953885507 translated_v:  0.5660279550957518
C=10
original_english: 0.8658714906417112 bad_english:  0.8435341466131907 all_v:  0.5762951087413385 translated_v:  0.566952729473916
C=100
original_english: 0.8362020944741534 bad_english:  0.8093179032976826 all_v:  0.5759562152490063 translated_v:  0.5664172903367819
C=500


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


original_english: 0.8272546234402852 bad_english:  0.794054979946524 all_v:  0.5754929086957044 translated_v:  0.5642969953775039
C=1000
original_english: 0.8244763814616757 bad_english:  0.7879832330659537 all_v:  0.5735529776272652 translated_v:  0.5612882456526525


На английском даже при обучении на дважды переведенных текстах лучше работает на оригинальных.На валидации от использования двойного перевода качество повысилось, но не сильно. 
На дважды переведенных английских работает сильно лучше, чем на переведенных с других языков
